In [1]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate

In [2]:
df = pd.read_csv('data.csv')
df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [8]:
df_title = pd.read_csv('movie_name.csv')
df_title.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [18]:
df_p = pd.pivot_table(df,values='rating',index='userId',columns='movieId')

KeyError: 'Rating'

In [12]:
f = ['count','mean']

df_movie_summary = df.groupby('movieId')['rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.8),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

df_cust_summary = df.groupby('userId')['rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.8),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index


In [6]:
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']][:], reader)
data.split(n_folds=3)
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

C:\Users\Anil Jacob\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\Anil Jacob\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9033
MAE:  0.6952
------------
Fold 2
RMSE: 0.9037
MAE:  0.6958
------------
Fold 3
RMSE: 0.9018
MAE:  0.6971
------------
------------
Mean RMSE: 0.9030
Mean MAE : 0.6960
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6951857923122954,
                             0.6958078980286507,
                             0.6971452801868843],
                            'rmse': [0.9033329025407383,
                             0.9037468506935813,
                             0.9018074057070544]})

In [10]:
df_398 = df[(df['userId'] == 398) & (df['rating'] == 5)]
df_398 = df_398.set_index('movieId')
df_398 = df_398.join(df_title)['title']
print(df_398)

movieId
173                                        Species (1995)
1197                                         Crash (1996)
1380                                 Shooting Fish (1997)
1923                                          Clue (1985)
2301                                     Excalibur (1981)
2916               Quatermass 2 (Enemy from Space) (1957)
3361                             Longest Yard, The (1974)
3671                         War of the Roses, The (1989)
3745    Endurance: Shackleton's Legendary Antarctic Ex...
3785                               My First Mister (2001)
3793                                Female Trouble (1975)
3997                              The Monster Club (1981)
Name: title, dtype: object


In [13]:
user_398 = df_title.copy()
user_398 = user_398.reset_index()
user_398 = user_398[~user_398['movieId'].isin(drop_movie_list)]

data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

trainset = data.build_full_trainset()
svd.train(trainset)

user_398['Estimate_Score'] = user_398['movieId'].apply(lambda x: svd.predict(398, x).est)

user_398 = user_398.drop('movieId', axis = 1)

user_398 = user_398.sort_values('Estimate_Score', ascending=False)
print(user_398.head(10))

C:\Users\Anil Jacob\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


      index                           title  Estimate_Score
726     726    It Happened One Night (1934)        5.000000
734     734      Maltese Falcon, The (1941)        4.985986
725     725              Rear Window (1954)        4.984571
729     729       North by Northwest (1959)        4.975977
984     984              Raging Bull (1980)        4.970034
741     741       Gone with the Wind (1939)        4.906936
719     719  Philadelphia Story, The (1940)        4.904495
1001   1001            Graduate, The (1967)        4.903867
695     695           Godfather, The (1972)        4.895327
266     266             Pulp Fiction (1994)        4.876645


In [16]:
def recommend(movie_title, min_count):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df_title.index[df_title['title'] == movie_title][0])
    target = df_p[i]
    similar_to_target = df_p.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'title', 'count', 'mean']]
    print(corr_target[corr_target['count']>min_count][:10].to_string(index=False))

In [17]:
recommend("Ace Ventura: When Nature Calls (1995)", 0)

For movie (Ace Ventura: When Nature Calls (1995))
- Top 10 movies recommended based on Pearsons'R correlation - 


NameError: name 'df_p' is not defined